In [2]:
import pandas as pd
import numpy as np

from IPython.display import clear_output

In [85]:
df = pd.read_csv('../data/league_discord.csv', index_col='Unnamed: 0')

In [86]:
df

,name,content,timestamp
0,Ziul#2536,<:wutmo:513395072025493505>,2020-11-01T20:36:44.881000+00:00
1,_ItzChris_#7028,<:dogeKek:639726958091173910>,2020-11-01T20:36:59.732000+00:00
2,HeapsOfHamood#3085,https://cdn.discordapp.com/attachments/3660512...,2020-11-01T20:37:00.621000+00:00
3,_ItzChris_#7028,Lmfao,2020-11-01T20:37:09.857000+00:00
4,Ziul#2536,"bruh, the best teemo player from my country is...",2020-11-01T20:37:41.427000+00:00
...,...,...,...
4035,consolidation/p&L/ratio/tax#7495,<@!411507772094414850> https://media.discordap...,2020-10-01T06:13:04.420000+00:00
4036,Strike345#5659,should i pick viktor or ryze?,2020-10-01T06:13:10.040000+00:00
4037,Strike345#5659,i picked viktor <:WEIRD:665202887009501192>,2020-10-01T06:13:25.493000+00:00
4038,Digitally#2636,I mostly get 10/8 and still lose something lik...,2020-10-01T06:05:48.131000+00:00


In [87]:
df = df.loc[:, ['content']]

In [88]:
df = df.drop_duplicates()

In [89]:
df = df.reindex(columns=df.columns.to_list() + ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

In [90]:
# Get rid of Emotes for Discord Messages
df.loc[:, 'content'] = df['content'].str.replace(r'<.*>', '')

# Get rid of urls
df.loc[:, 'content'] = df['content'].str.replace(r'http\S+', '')

In [92]:
# Get rid of empty Strings
df = df[df['content'] != ""]

In [105]:
df = df.reset_index(drop=True)

In [110]:
current_index = 151
stop = 0

In [111]:
while current_index < len(df) and not stop:
    print(f"Current Index: {current_index}")
    print(df.loc[current_index, 'content'])
    
    print("Labeling Toxic:")
    toxic = input()
    df.loc[current_index, 'toxic'] = 0 if not toxic else int(toxic)
    print("Labeling Severe Toxic:")
    severe_toxic = input()
    df.loc[current_index, 'severe_toxic'] = 0 if not severe_toxic else int(severe_toxic)
    print("Labeling Obscene:")
    obscene = input()
    df.loc[current_index, 'obscene'] = 0 if not obscene else int(obscene)
    print("Labeling Threat:")
    threat = input()
    df.loc[current_index, 'threat'] = 0 if not threat else int(threat)
    print("Labeling Insult:")
    insult = input()
    df.loc[current_index, 'insult'] = 0 if not insult else int(insult)
    print("Labeling Identity Hate:")
    identity_hate = input()
    df.loc[current_index, 'identity_hate'] = 0 if not identity_hate else int(identity_hate)


    print("Stop?")
    stop = input()
    stop = 0 if not stop else int(stop)
    
    current_index += 1
    
    clear_output()
    
stop = 0

In [116]:
labeled_df = df[~df['toxic'].isnull()]

In [117]:
labeled_df

,content,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,\nreal faker tweet,0.0,0.0,0.0,0.0,0.0,0.0
1,Lmfao,0.0,0.0,0.0,0.0,0.0,0.0
2,"bruh, the best teemo player from my country is...",0.0,0.0,0.0,0.0,0.0,0.0
3,me and nunu tiered 3 simped for adc,0.0,0.0,0.0,0.0,0.0,0.0
4,yeah but i think since seraphine was released ...,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
196,lux support,0.0,0.0,0.0,0.0,0.0,0.0
197,bois theres a fucking electrical storm going o...,0.0,0.0,0.0,0.0,0.0,0.0
198,Yo Riot wtf,0.0,0.0,0.0,0.0,0.0,0.0
199,55% WR,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
labeled_df.to_csv('./data/labeled_league_discord.csv')